In [1]:
import instaloader
from decouple import config
from IPython.display import clear_output
import pandas as pd
import time
from os.path import exists

C:\Users\Razer\anaconda3\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [2]:
accounts = [i for i in range(1,6)]
account = accounts[0]
USER = config(f'USER{account}',default='')
PASSWORD = config(f'PASSWORD{account}',default='')
print(USER)
L = instaloader.Instaloader()
L.login(USER, PASSWORD)

wesleyongs


# Competitors followers

## followers

https://gephi.org/users/supported-graph-formats/csv-format/#:~:text=Gephi%20supports%20CSV%20files%20that,by%20single%20or%20double%20quotes.

In [15]:
# competitors = ['kayla.crystals']

# for username in competitors:
#     pd.read_csv(f"{username}.csv")['followers'].apply(lambda x: x.split(" ")[1]).to_csv(f'{username}.csv')

In [28]:
competitors = [
    'loveforcrystals', 'tamzacrystals', 'gemstorysg',
    'glacecrystals', 'ohmou.sg', 'crystalmojo', 'covetcrystalsjewelry',
    'fated.itis', 'illa_nocte', 'kayla.crystals','soulalign'
]

for username in competitors:
    
    print(f"Doing {username}")
    path = 'competitors_followers'
    file_name = f'{path}/{username}.csv'
    if exists(file_name):continue
        
    profile = instaloader.Profile.from_username(L.context, username)
    followers = profile.get_followers()
    follower_count = profile.followers
    follower_list = []
    
    if exists(file_name):follower_list=pd.read_csv(file_name)['followers'].to_list()
    else:follower_list = []
    if len(follower_list)==follower_count:continue

    for idx, follower in enumerate(followers):
        
        if follower.username in follower_list:continue
        clear_output(wait=True)
        print(f"{idx}/{follower_count} done for {username}")
        follower_list.append(follower.username)
        pd.DataFrame(data={'followers':follower_list}).to_csv(file_name)

Doing loveforcrystals
Doing tamzacrystals
Doing gemstorysg
Doing glacecrystals
Doing ohmou.sg
Doing crystalmojo
Doing covetcrystalsjewelry
Doing fated.itis
Doing illa_nocte
Doing kayla.crystals
Doing soulalign


## competitor details

In [3]:
competitors = [
    'loveforcrystals', 'tamzacrystals', 'gemstorysg',
    'glacecrystals', 'ohmou.sg', 'crystalmojo', 'covetcrystalsjewelry',
    'fated.itis', 'illa_nocte', 'kayla.crystals','soulalign'
]

for username in competitors:
    
    path = 'competitors_details/2.0'
    file_name = f"{path}/{username}.csv"

    profile = instaloader.Profile.from_username(L.context, username)
    posts = instaloader.Profile.from_username(L.context, username).get_posts() 
    total_posts = profile.mediacount

    # create dataset
    if exists(file_name):data = pd.read_csv(file_name).iloc[:,1:].to_dict('list')
    else:data = {'index':[], 'date':[], 'caption':[], 'comments':[], 'likes':[]}
    data = {'index':[], 'date':[], 'caption':[], 'comments':[], 'likes':[]}
    
    # Loop the followers 
    for idx, post in enumerate(posts):

        if idx in data['index']:continue
        data['index'].append(idx)
        data['date'].append(post.date_local)
        data['caption'].append(post.caption)
        data['comments'].append(post.comments)
        data['likes'].append(post.likes)
        pd.DataFrame(data=data).to_csv(file_name)
        clear_output(wait=True)
        print(f"{idx}/{total_posts} done for {username}")

520/521 done for soulalign


#  followers of JS


In [9]:
js_followers = pd.read_csv('followers.csv')['0']
edges = pd.read_csv('edges.csv')
data = edges[['from','to']].to_dict('list')
edges['from'].value_counts()
x=edges.groupby('from')[['to']].count().reset_index().sort_values(by='to')
print("accounts scraped (not private or rejected):", x[x['to']>1]['to'].count())

accounts scraped (not private or rejected): 5


In [ ]:
# data = {'from':[], 'to':[]}
exceptions = {}
account_pointer = 0

# stats
privates = 0
no_posts = 0
bad_ratio = 0

for idx, username in enumerate(js_followers):

    if username in data['from']:continue
    if idx < 850:continue
    if idx == 1000:break

    # Loader
    try:
        profile = instaloader.Profile.from_username(L.context, username)
        following = profile.get_followees()
    except Exception as e:
        if e in exceptions.keys():exceptions[e]+=1
        else:exceptions[e]=1

    # variables
    is_private = profile.is_private
    no_posts = profile.mediacount
    ratio = profile.followees/(profile.followers or not profile.followers)
    print(f'doing {username} who is following {profile.followees} accounts')
    print(f"Private:{is_private} Posts:{no_posts} Ratio:{round(ratio,1)}")

    # checks
    if is_private:privates+=1
    if no_posts==0:no_posts+=1
    if ratio>10:bad_ratio+=1
    if no_posts==0 or ratio > 10:
        data['from'].append(username)
        data['to'].append("jam.stones")
        continue

    # collect data
    for follower in following:
        data['from'].append(username)
        data['to'].append(follower.username)

    # add JS edge
    data['from'].append(username)
    data['to'].append("jam.stones")

    # save file
    pd.DataFrame(data).to_csv("edges.csv")

    print(f'finished js follower {idx}/{len(js_followers)}')
        

doing fishcrystals who is following 857 accounts
Private:False Posts:101 Ratio:0.3
finished js follower 861/13404
doing pammiethia who is following 1805 accounts
Private:True Posts:194 Ratio:7.3
finished js follower 862/13404
doing juuhnel who is following 687 accounts
Private:False Posts:10 Ratio:2.3
finished js follower 863/13404
doing carteeka_ who is following 675 accounts
Private:True Posts:2591 Ratio:1.1
finished js follower 864/13404
doing crystal.wishlist_v who is following 296 accounts
Private:False Posts:35 Ratio:0.8
finished js follower 865/13404
doing crz.stars who is following 109 accounts
Private:False Posts:62 Ratio:0.7
finished js follower 866/13404
doing sc_vstd who is following 85 accounts
Private:True Posts:161 Ratio:0.7
finished js follower 867/13404
doing its_me_caren who is following 1785 accounts
Private:True Posts:527 Ratio:4.6
finished js follower 868/13404
doing et_ml_chaoahlian who is following 426 accounts
Private:True Posts:214 Ratio:3.1
finished js followe

JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:   12   12   12   12   12   12
 * 58712303d941c6855d4e888c5f0cd22f:   49   49   49   49   49   49
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do not use any Instagram App while Instaloader
is running.
The request will be retried in 527 seconds, at 16:15.
JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    8   10   12   12   12   12
 * 58712303d941c6855d4e888c5f0cd22f:   18   43   50   50   50   50
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, d

doing karynkhoo who is following 490 accounts
Private:True Posts:273 Ratio:2.6
finished js follower 871/13404


JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    8   11   13   13   13   13
 * 58712303d941c6855d4e888c5f0cd22f:   18   37   52   52   52   52
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do not use any Instagram App while Instaloader
is running.
The request will be retried in 8 seconds, at 16:16.
JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    7   10   13   13   13   13
 * 58712303d941c6855d4e888c5f0cd22f:   14   34   53   53   53   53
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do 

doing doreen.lim.311 who is following 57 accounts
Private:False Posts:0 Ratio:1.0
finished js follower 872/13404


...

JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    2    4    6    7    9   20
 * 58712303d941c6855d4e888c5f0cd22f:    6   11   17   21   27   59
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do not use any Instagram App while Instaloader
is running.
The request will be retried in 25 seconds, at 18:42.
JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    2    3    6    7    9   20
 * 58712303d941c6855d4e888c5f0cd22f:    7   10   18   21   28   60
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do

doing khyangela who is following 238 accounts
Private:True Posts:9 Ratio:4.0
finished js follower 927/13404


JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    3    4    7    7   10   21
 * 58712303d941c6855d4e888c5f0cd22f:    9   11   20   20   30   62
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do not use any Instagram App while Instaloader
is running.
The request will be retried in 8 seconds, at 18:42.
JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    3    3    7    7   10   21
 * 58712303d941c6855d4e888c5f0cd22f:   10   11   21   21   31   63
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do 

doing joanneisme2022 who is following 32 accounts
Private:False Posts:9 Ratio:1.6
finished js follower 928/13404


JSON Query to graphql/query: 429 Too Many Requests [retrying; skip with ^C]
Number of requests within last 10/11/20/22/30/60 minutes grouped by type:
                              other:    4    4    8    8   11   22
 * 58712303d941c6855d4e888c5f0cd22f:   12   12   23   23   33   65
Instagram responded with HTTP error "429 - Too Many Requests". Please
do not run multiple instances of Instaloader in parallel or within
short sequence. Also, do not use any Instagram App while Instaloader
is running.
The request will be retried in 517 seconds, at 18:51.


In [41]:
print(privates, no_posts, bad_ratio)
print(data)

5 0 4
{'from': ['agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agnestankimpheng', 'agn